In [ ]:
import os
import sys
import logging
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# API 키 설정
# os.environ["OPENAI_API_KEY"] = "당신의 API 키를 입력하세요"
# LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

In [ ]:
# === 문서 로드 및 인덱스 생성 (RAG) ===
# 문서 폴더 내에는 아래 데이터가 포함되어 있다고 가정.
# 1. 합격 자소서 데이터, 2. 합격자 면접 데이터, 3. 업계 동향 데이터, 4. 업계 구인 데이터

# 문서 로드를 위한 패키지
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Document
from llama_index.core import GPTVectorStoreIndex

# 문서 로드 및 인덱스 생성
# documents = SimpleDirectoryReader("데이터 파일들의 경로를 입력하세요").load_data()
index = GPTVectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

In [10]:
# 참고 자료 검색: 자소서 분석 및 피드백에 유용한 자료를 외부 문서에서 추출
reference_query = (
    "합격 자소서, 합격자 면접, 업계 동향, 구인 데이터를 참고하여, "
    "자소서 작성 피드백에 유용한 핵심 인사이트를 간략히 요약해줘."
)
reference_result = query_engine.query(reference_query)

# 지원 기업 관련 산업 뉴스 데이터 추출 (지원 기업 및 관련 키워드 활용)
company = "A기업"
industry_news_query = (
    f"{company} 관련 산업 뉴스, 최신 동향, 전략 및 이슈"
)
industry_news_result = query_engine.query(industry_news_query)


# 두 결과를 결합하여 참고 자료로 활용
combined_reference = f"{reference_result}\n\n[산업 뉴스 참고]:\n{industry_news_result}"


In [11]:
## 여기부터 실행

# === 수정된 분석 체인 프롬프트 템플릿 (회사, 직무 정보 포함 및 적합성 체크) ===
analysis_prompt = PromptTemplate(
    input_variables=["자소서내용", "참고자료", "회사", "직무"],
    template="""
사용자가 입력한 자소서 내용은 다음과 같습니다:
--------------------------------------
{자소서내용}
--------------------------------------
아래의 참고 자료는 합격 자소서, 면접 데이터, 업계 동향, 구인 데이터 및 지원 기업 관련 산업 뉴스에서 추출된 인사이트입니다:
--------------------------------------
{참고자료}
--------------------------------------
또한, 사용자가 지원하는 회사는 "{회사}"이며 희망 직무는 "{직무}"입니다.
위 정보를 종합하여, 다음 사항들을 분석해줘:
1. 자소서에 나타난 강점과 약점을 각각 3가지씩 구체적으로 제시
2. 자소서 내용이 지원하는 회사와 희망 직무에 얼마나 부합하는지 평가하고, 개선이 필요한 부분(예: 지원 동기, 관련 경험, 역량 표현 등)을 구체적으로 지적해줘.
예시) 강점: [문제 해결 능력, 팀워크 등] / 약점: [경험 부족, 구체성 부족 등] / 적합성 평가: [지원 동기 보완 필요 등]
"""
)

In [12]:
# === 조언 체인 프롬프트 템플릿 (회사, 직무 정보 추가) ===
advice_prompt = PromptTemplate(
    input_variables=["분석결과", "회사", "직무"],
    template="""
아래 분석 결과와 지원 정보(지원 회사 및 직무)를 참고하여, 자소서 개선 및 보완을 위한 구체적인 조언을 작성해줘.
분석 결과:
--------------------------------------
{분석결과}
--------------------------------------
지원 정보:
- 지원하는 회사: {회사}
- 지원하는 직무: {직무}

조언은 실제 자소서에 적용 가능한 수정 방안과 개선 방향을 상세하게 제시하고, 지원 회사와 직무에 맞는 어필 포인트를 반영해줘.
"""
)

In [13]:
# === 자기소개서 예시 출력 프롬프트 템플릿 (회사, 직무 정보 포함) ===
example_prompt = PromptTemplate(
    input_variables=["조언결과", "회사", "직무"],
    template="""
아래 조언 결과를 바탕으로, 실제 지원자가 제출할 수 있는 자기소개서 예시를 작성해줘.
자기소개서는 다음과 같은 구조를 갖추어야 합니다:
1. **지원동기 및 비전:** 지원 동기, 본인의 비전, 그리고 지원 회사({회사}) 및 직무({직무})와의 시너지를 명확히 서술
2. **경험 및 역량:** 학습, 동아리/프로젝트 경험, 보유 기술 및 역량을 구체적으로 기술
3. **강점 및 보완 사항:** 분석된 강점을 부각시키고, 약점을 개선하기 위한 구체적인 실행 계획 제시
4. **향후 계획:** 입사 후 성장 방향과 목표

각 항목별로 소제목을 붙여 자기소개서 형식(각 섹션별로 구분)으로 작성해줘.
아래 조언 결과:
--------------------------------------
{조언결과}
--------------------------------------
자기소개서 예시:
"""
)

In [14]:
# 체인 생성: 분석 체인, 조언 체인, 예시 출력 체인
analysis_chain = LLMChain(llm=llm, prompt=analysis_prompt)
advice_chain = LLMChain(llm=llm, prompt=advice_prompt)
example_chain = LLMChain(llm=llm, prompt=example_prompt)

/var/folders/dn/cz4fv22x5s7bp8_g1_qj6k_w0000gn/T/ipykernel_3542/2145800932.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  analysis_chain = LLMChain(llm=llm, prompt=analysis_prompt)


In [ ]:
# === 사용자의 입력 ===
user_self_intro = """
반감습니다. 저는 경영학을 전공했고 TESAT 등의 다양한 경영 관련 자격증을 취득했습니다.
특히 회사 투어를 통해 직접 기업 문화를 경험하고, 그 과정에서 경영에 대한 열정을 더욱 키웠습니다.
또한 학부 시절 동아리 활동을 통해 리더십과 팀워크 능력을 키웠습니다.
이러한 경험을 밑거름으로 회사에서의 성장과 기여를 꿈꾸고 있습니다.
실무 경험은 부족하지만, 열정과 노력으로 보완하고 성장하겠습니다.
"""

# 추가 입력: 지원하는 회사와 직무
company = "sk쉴더스"
job_position = "소프트웨어 엔지니어"

In [16]:
# 1. 분석 체인 실행: 자소서 내용과 참고 자료, 지원 정보를 기반으로 강점/약점 및 적합성 평가
analysis_result = analysis_chain.run(
    자소서내용=user_self_intro,
    참고자료=reference_result,
    회사=company,
    직무=job_position
)
print("=== 분석 결과 ===")
print(analysis_result)


/var/folders/dn/cz4fv22x5s7bp8_g1_qj6k_w0000gn/T/ipykernel_3542/3352268720.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  analysis_result = analysis_chain.run(


=== 분석 결과 ===
1. **자소서에 나타난 강점과 약점**

   **강점:**
   - **경영학 전공과 자격증:** 경영학을 전공하고 관련 자격증을 취득한 점은 이론적인 지식과 기본적인 경영 능력을 갖추고 있음을 보여줍니다. 이는 소프트웨어 엔지니어로서 비즈니스 이해도를 높이는 데 도움이 될 수 있습니다.
   - **기업 문화 경험:** 회사 투어를 통해 기업 문화를 경험한 점은 해당 회사에 대한 이해도를 높이고, 기업에 대한 적합성을 보여주는 좋은 예입니다. 이는 지원하는 회사인 SK쉴더스에 대한 관심과 열정을 나타냅니다.
   - **리더십과 팀워크:** 동아리 활동을 통해 리더십과 팀워크 능력을 키운 점은 소프트웨어 엔지니어로서 협업이 중요한 환경에서 긍정적인 요소가 될 수 있습니다.

   **약점:**
   - **실무 경험 부족:** 자소서에서 실무 경험이 부족하다고 언급한 점은 지원 직무에 필수적인 기술적 역량이나 경험이 결여되어 있음을 나타냅니다. 소프트웨어 엔지니어는 실무적인 경험이 중요하기 때문에 이는 단점으로 작용할 수 있습니다.
   - **구체성 부족:** 자소서 내용에서 경영에 대한 열정이나 경험이 구체적으로 어떤 방식으로 소프트웨어 개발에 연결될 수 있는지에 대한 설명이 부족합니다. 이는 지원자의 역량을 충분히 전달하지 못하는 요소가 됩니다.
   - **지원 동기의 부족:** SK쉴더스와의 구체적인 연결 고리가 부족하여, 왜 이 회사에서 일하고 싶은지에 대한 강력한 동기가 드러나지 않습니다. 이는 지원의 진정성을 약화시킬 수 있습니다.

2. **적합성 평가 및 개선 필요 사항**

   **적합성 평가:** 
   - 자소서의 내용은 경영학 전공 및 리더십 경험 등에서 긍정적인 요소를 보이나, 소프트웨어 엔지니어라는 직무와의 직접적인 연결이 부족합니다. SK쉴더스의 사업 영역과 소프트웨어 엔지니어로서의 역할을 고려할 때, 기술적 역량과 실무 경험이 더 부각되어야 합니다.

   **개선이 필요한 부분:**
   - *

In [17]:
# 2. 조언 체인 실행: 분석 결과와 지원 정보를 기반으로 개선 조언 생성
advice_result = advice_chain.run(
    분석결과=analysis_result,
    회사=company,
    직무=job_position
)
print("\n=== 조언 결과 ===")
print(advice_result)


=== 조언 결과 ===
자기소개서 개선을 위해 다음과 같은 구체적인 조언을 드립니다. SK쉴더스의 소프트웨어 엔지니어 직무에 적합하도록 내용을 보강하고, 지원자의 강점을 더욱 부각시킬 수 있는 방향으로 수정해보세요.

### 1. 지원 동기 및 회사 이해도 강화
- **회사 비전 및 목표와의 연계:** SK쉴더스의 비전이나 목표, 특히 소프트웨어 엔지니어 직무와 관련된 사업 영역을 명확히 언급하세요. 예를 들어, “SK쉴더스가 추구하는 안전하고 스마트한 사회 구축에 기여하고 싶습니다. 이를 위해 제가 보유한 경영학적 지식을 활용하여 소프트웨어 개발 과정에서 사용자 경험을 최적화하고, 비즈니스 요구사항을 효과적으로 반영할 수 있습니다.”와 같은 구체적인 연결 고리를 만들어 보세요.
- **회사의 최근 뉴스나 프로젝트 언급:** SK쉴더스의 최근 프로젝트나 기술 혁신에 대해 언급하고, 그에 대한 본인의 관심이나 열정을 표현하세요. 예를 들어, “최근 SK쉴더스가 진행한 XYZ 프로젝트에 대한 기사를 읽고 큰 영감을 받았습니다. 이 프로젝트에서 제가 기여할 수 있는 부분을 구체적으로 상상해보았습니다.”와 같이 구체화하면 좋습니다.

### 2. 실무 경험 강조
- **프로젝트 경험 추가:** 학부 시절에 진행한 소프트웨어 개발 프로젝트나 팀 프로젝트를 구체적으로 기술하세요. 예를 들어, “대학에서 진행한 ‘ABC 애플리케이션 개발 프로젝트’에서 팀 리더로서 전체 프로젝트를 관리하며, Java와 SQL을 활용하여 데이터베이스를 설계하고 구현하였습니다. 이 경험을 통해 문제 해결 능력을 키웠고, 개발 과정에서 발생하는 다양한 이슈를 해결하는 방법을 배웠습니다.”와 같이 구체적인 사례를 통해 실무 경험을 강조합니다.
- **인턴 경험 언급:** 만약 인턴십 경험이 있다면, 그 경험을 구체적으로 서술하여 해당 직무와의 연관성을 명확히 하세요. 예를 들어, “XYZ 회사에서 소프트웨어 개발 인턴으로 근무하며, 실제 환경에서 코드를 작성하고, 팀원들과 협업하여 기능 개선 

In [18]:
# 3. 예시 출력 체인 실행: 조언 결과와 지원 정보를 반영한 자기소개서 예시 생성
example_self_intro = example_chain.run(
    조언결과=advice_result,
    회사=company,
    직무=job_position
)
print("\n=== 자기소개서 예시 ===")
print(example_self_intro)


=== 자기소개서 예시 ===
# 자기소개서

## 1. 지원 동기 및 비전
저는 SK쉴더스의 소프트웨어 엔지니어 직무에 지원하게 된 이유는, 이 회사가 추구하는 '안전하고 스마트한 사회'를 구현하는 데 기여하고 싶기 때문입니다. 특히, 최근 SK쉴더스가 발표한 ‘스마트 보안 시스템’ 프로젝트에 대한 기사를 읽고 큰 영감을 받았습니다. 이 프로젝트에서 저의 경영학적 지식을 활용하여 사용자 경험을 최적화하고, 비즈니스 요구사항을 효과적으로 반영하는 개발을 통해 기여할 수 있겠다는 확신을 가졌습니다. 제 비전은 고객이 안전하게 생활할 수 있는 환경을 소프트웨어로 구현하는 것이며, SK쉴더스와 함께 그 비전을 실현하고자 합니다.

## 2. 경험 및 역량
대학 시절, ‘ABC 애플리케이션 개발 프로젝트’에서 팀 리더로 참여하여 Java와 SQL을 활용하여 데이터베이스를 설계하고 구현하였습니다. 이 프로젝트에서는 기획 단계에서부터 최종 배포까지 모든 과정을 관리하였으며, 문제 해결 능력을 키우는 데 큰 도움이 되었습니다. 또한, XYZ 회사에서 소프트웨어 개발 인턴으로 근무하며 실제 환경에서 코드를 작성하고 팀원들과 협업하여 기능 개선 작업을 수행했습니다. 이러한 경험을 통해 협업의 중요성과 실제 개발 환경에서의 문제 해결 능력을 배양할 수 있었습니다.

저는 Python과 Java에 능숙하며, Git을 이용한 버전 관리 및 Agile 개발 방법론에 대한 이해도 가지고 있습니다. 특히, Python을 활용한 데이터 분석 경험은 소프트웨어 엔지니어로서 고객의 요구를 분석하고 맞춤형 솔루션을 제공하는 데 큰 자산이 될 것입니다.

## 3. 강점 및 보완 사항
제 강점은 뛰어난 문제 해결 능력과 팀워크입니다. 과거 프로젝트에서 팀원 간의 의견 차이로 인해 일정이 지연될 위기에 처했을 때, 중재 역할을 맡아 팀원들의 의견을 조율하고 새로운 일정을 제안하여 성공적으로 프로젝트를 완료한 경험이 있습니다. 이러한 경험들은 저에게 협업의 중요성과 다양한 의견을 수렴하여 최적의 